# 데이터 로드 

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ./


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!mkdir data 
%cd data 

!kaggle competitions download -c tabular-playground-series-apr-2022
!unzip tabular-playground-series-apr-2022.zip 

/content/data
 94% 161M/171M [00:06<00:00, 28.3MB/s]
100% 171M/171M [00:06<00:00, 26.6MB/s]
Archive:  tabular-playground-series-apr-2022.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
  inflating: train_labels.csv        


In [ ]:
from glob import glob 
from tqdm import tqdm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import os 

In [ ]:
%cd data
file_dirs = glob('*.csv')

df = []
for dir in file_dirs:
  exec(f"{dir.split('.')[0]}=pd.read_csv('{dir}')")
  df.append(dir.split('.')[0])

# 전처리 

In [ ]:
class preprocess():
  def __init__(self,input_df,test_df):
    self.input_df = self.init_pre(input_df)
    self.train_df, self.valid_df = self.train_valid_split(self.input_df)
    self.minmax_value = self.make_minmax(self.train_df)
    self.test_df = self.init_pre(test_df)

  #기본 전처리 모듈
  def init_pre(self,input_df:pd.DataFrame):
    input_df = input_df.drop(columns = ['sequence','subject','step'])
    input_df = np.array(input_df).reshape(-1,60,13)
    return input_df 
  
  def train_valid_split(self,input_df):
    id = int(len(input_df)*0.1)
    train_df = input_df[:id*9]
    valid_df = input_df[id*9:]
    return train_df, valid_df 
  
  def make_minmax(self,input_df):
    # min_value = np.min(input_df,axis=0)
    # max_value = np.max(input_df,axis=0)
    min_value = np.percentile(input_df,25)
    max_value = np.percentile(input_df,75)
    scale_value = {}
    scale_value['min'] = min_value
    scale_value['max'] = max_value
    return scale_value
  
  def scaler(self,input_df):
    train_min_value = self.minmax_value['min']
    train_max_value = self.minmax_value['max']
    return_value = (input_df-train_min_value)/(train_max_value-train_min_value)
    return_value = return_value.astype(np.float16)
    np.random.shuffle(return_value)
    return return_value 

  # return  
  def __call__(self):
    self.train_df = self.scaler(self.train_df)
    self.valid_df = self.scaler(self.valid_df)
    self.test_df = self.scaler(self.test_df)
    return self.train_df,self.valid_df,self.test_df

def make_labels(labels):
  id = int(len(labels)*0.1)

  labels = np.array(labels.drop(columns=['sequence']))
  train_y = labels[:id*9]
  valid_y = labels[id*9:]
  return train_y, valid_y 

train_df,valid_df,test_df = preprocess(train,test)()
train_y, valid_y = make_labels(train_labels)

# 모델 

In [ ]:
!pip install tensorflow-addons
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.layers import Input,Dense,LSTM,Bidirectional,Conv2D
from tensorflow.keras import optimizers
import tensorflow_addons as tfons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 34.4 MB/s 


## 하이퍼 파라미터 튜닝 

In [ ]:
opt = {} 
opt['input_shape'] = train_df.shape[1:]
opt['batch_size'] = 32 
opt['epochs'] = 5
opt['init_lr'] = 0.0005
opt['optimizer'] = optimizers.Adam(learning_rate = opt['init_lr'])

In [121]:
def encoding_block(node,input_layer):
  x = input_layer 
  x = layers.Dropout(0.2)(x)
  x = Bidirectional(LSTM(node,return_sequences=True))(x)
  x = layers.Normalization()(x)
  return x 

def make_model(opt):
  inputs = Input(opt['input_shape'])
  x = inputs
  for i in [128,32,2]:
    x = encoding_block(i,x)
  x = layers.Flatten()(x)
  x = layers.Dropout(0.2)(x)
  x = Dense(10,activation='relu')(x)
  x = Dense(2,activation = 'softmax')(x)
  model = keras.Model(inputs,x)
  
  model.compile(optimizer = opt['optimizer'],loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
  history = model.fit(train_df,train_y,batch_size=opt['batch_size'],validation_data=(valid_df,valid_y),epochs=opt['epochs'],verbose=1)
  return history,model 

In [78]:
batch_size = {} 
batch_size['64'] = 64 
batch_size['32'] = 32 

init_lr = {}
init_lr['0.0005'] = 0.0005
init_lr['0.001'] = 0.001 

opt_dict = {}
opt_dict['Radam'] = tfons.optimizers.RectifiedAdam()
opt_dict['Adamw'] = tfons.optimizers.AdamW(weight_decay = 1e-4)
opt_dict['Adam'] = optimizers.Adam() 
opt_dict['RMSprop'] = optimizers.RMSprop()

schedules = {} 
schedules['exp'] = optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0001,decay_steps=1000,decay_rate=0.96)


In [79]:
result_save = [] 
for lr_name,lr in init_lr.items():
  for sch_name, schedule in schedules.items():
    for opt_name,optimizer in opt_dict.items():
      for t_batch,n_batch in batch_size.items():
        schedule.initial_learning_rate = lr 
        optimizer.learning_rate=schedule
        opt['optimizer'] = optimizer
        opt['batch_size'] = n_batch 
        condition = [lr_name,sch_name,t_batch,opt_name]
        print(condition)
        history = make_model(opt)
        condition = [lr_name,sch_name,t_batch,opt_name,history.history['accuracy'][-1]]
        result_save.append(condition)

['0.0005', 'exp', '64', 'Radam']
Epoch 1/5
366/366 [==============================] - 22s 35ms/step - loss: 0.6947 - accuracy: 0.4976 - val_loss: 0.6941 - val_accuracy: 0.5035
Epoch 2/5
366/366 [==============================] - 10s 27ms/step - loss: 0.6928 - accuracy: 0.5067 - val_loss: 0.6945 - val_accuracy: 0.5019
Epoch 3/5
366/366 [==============================] - 10s 28ms/step - loss: 0.6912 - accuracy: 0.5187 - val_loss: 0.6947 - val_accuracy: 0.4962
Epoch 4/5
366/366 [==============================] - 10s 28ms/step - loss: 0.6892 - accuracy: 0.5270 - val_loss: 0.6947 - val_accuracy: 0.5115
Epoch 5/5
366/366 [==============================] - 10s 27ms/step - loss: 0.6861 - accuracy: 0.5397 - val_loss: 0.6989 - val_accuracy: 0.4854
['0.0005', 'exp', '32', 'Radam']
Epoch 1/5
731/731 [==============================] - 31s 29ms/step - loss: 0.6942 - accuracy: 0.5011 - val_loss: 0.6934 - val_accuracy: 0.4896
Epoch 2/5
731/731 [==============================] - 19s 27ms/step - loss: 0

In [80]:
result_save = pd.DataFrame(result_save)
result_save.columns = ['lr','schedule','batch','opt','acc']

In [85]:
result_save.sort_values(by=['acc'],ascending=False)

,lr,schedule,batch,opt,acc
6,0.0005,exp,64,RMSprop,0.565186
4,0.0005,exp,64,Adam,0.554143
14,0.001,exp,64,RMSprop,0.548536
7,0.0005,exp,32,RMSprop,0.545497
0,0.0005,exp,64,Radam,0.539719
2,0.0005,exp,64,Adamw,0.538820
15,0.001,exp,32,RMSprop,0.536638
5,0.0005,exp,32,Adam,0.528762
1,0.0005,exp,32,Radam,0.527050
3,0.0005,exp,32,Adamw,0.521315


## 튜닝 결과로 학습 

In [122]:
opt['batch_size'] = 64 
opt['epochs'] = 10
opt['init_lr'] = 0.0005 

opt_dict = {}
opt_dict['Radam'] = tfons.optimizers.RectifiedAdam()
opt_dict['Adamw'] = tfons.optimizers.AdamW(weight_decay = 1e-4)
opt_dict['Adam'] = optimizers.Adam() 
opt_dict['RMSprop'] = optimizers.RMSprop()

model_save = [] 
result_save = [] 
for optimizer in list(opt_dict.values()):
  opt['optimizer'] = optimizer 
  history,model = make_model(opt)
  
  result_save.append(history.history['accuracy'][-1])
  model_save.append(model)

Epoch 1/10
366/366 [==============================] - 24s 41ms/step - loss: 0.6969 - accuracy: 0.4975 - val_loss: 0.6934 - val_accuracy: 0.4973
Epoch 2/10
366/366 [==============================] - 10s 28ms/step - loss: 0.6933 - accuracy: 0.5009 - val_loss: 0.6934 - val_accuracy: 0.4939
Epoch 3/10
366/366 [==============================] - 10s 28ms/step - loss: 0.6932 - accuracy: 0.4945 - val_loss: 0.6932 - val_accuracy: 0.4942
Epoch 4/10
366/366 [==============================] - 10s 28ms/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6930 - val_accuracy: 0.4950
Epoch 5/10
366/366 [==============================] - 10s 28ms/step - loss: 0.6932 - accuracy: 0.4991 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 6/10
366/366 [==============================] - 11s 29ms/step - loss: 0.6931 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.4942
Epoch 7/10
366/366 [==============================] - 10s 28ms/step - loss: 0.6934 - accuracy: 0.5010 - val_loss: 0.6931 - val_accuracy:

In [106]:
list(opt_dict.values())